In [1]:
import pandas as pd
import syft as sy
import numpy as np
from syft.core.adp.entity import Entity

In [2]:
data = pd.read_csv("../../trade_demo/datasets/ca - feb 2021.csv", low_memory=False)[0:10]
data.head()

,Classification,Year,Period,Period Desc.,Aggregate Level,Is Leaf Code,Trade Flow Code,Trade Flow,Reporter Code,Reporter,...,Partner,Partner ISO,Commodity Code,Commodity,Qty Unit Code,Qty Unit,Qty,Netweight (kg),Trade Value (US$),Flag
0,HS,2021,202102,February 2021,4,0,1,Imports,124,Canada,...,"Other Asia, nes",NaN,6117,"Clothing accessories; made up, knitted or croc...",0,NaN,NaN,NaN,9285,0
1,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Egypt,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,116604,0
2,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Kingdom,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,1495175,0
3,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,United Rep. of Tanzania,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,2248,0
4,HS,2021,202102,February 2021,2,0,1,Imports,124,Canada,...,Singapore,NaN,18,Cocoa and cocoa preparations,0,NaN,NaN,0.0,47840,0


In [3]:
# Let's login into the domain
ca = sy.login(email="info@openmined.org", password="changethis", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!


In [7]:
ca.datasets[0]['Canada Trade']

<TensorWrappedSingleEntityPhiTensorPointer -> canada:722bb07654644897a71eb86134a387e2>

In [5]:
# We will upload only the first few rows
# All these three columns are of `int` type

# NOTE: casting this tensor as np.int32 is REALLY IMPORTANT. We need to create flags for this or something
canada_trade = ((np.array(list(data['Trade Value (US$)'])) / 100000)[0:10]).astype(np.int32)
trade_partners = ((list(data['Partner'])))[0:10]

entities = list()
for i in range(len(trade_partners)):
    entities.append(Entity(name=trade_partners[i]))

# Upload a private dataset to the Domain object, as the root owner
sampled_canada_dataset = sy.Tensor(canada_trade)
sampled_canada_dataset.public_shape = sampled_canada_dataset.shape
sampled_canada_dataset = sampled_canada_dataset.private(0, 3, entity=entities[0]).tag("trade_flow")

In [9]:
ca.load_dataset(assets={"Canada Trade": sampled_canada_dataset, "More Trade": sampled_canada_dataset},
                name="Sample Dataset",
                description="A nice description.")

Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [11]:
ca.datasets[3]

,name,description,tags,id,data
0,Sample Dataset,A nice description.,[],749a1a42-28f5-4a6e-baaf-8eddb861adb4,"[{'name': 'Canada Trade', 'id': 'a1dbe15b-59ea..."


In [7]:
datasets = ca.datasets

In [11]:
datasets[0].data

[{'name': 'Canada Trade',
  'id': '34cb7604-ea31-4c42-b33c-1992dc2aa09e',
  'dtype': 'Tensor',
  'shape': '(10,)'}]

In [191]:
assets = {"Canada Trade": sampled_canada_dataset}
name = "Sample Dataset"
description="""A collection of reports from Canada's statistics bureau about how much it thinks it imports and exports from other countries."""


import ipywidgets as widgets
from IPython.display import HTML
# table = [["Asset Names"],
#          ["Canada Trade"],
# #          ["Moon",1737,73.5],
# #          ["Mars",3390,641.85]
#         ]
# table = widgets.HTML(
#     value=str(HTML(tabulate.tabulate(table, tablefmt='html')).data),
#     placeholder='',
#     description='',
# )

# table = [widgets.Label('ASSETS')] + [widgets.Label(str(i)) for i in assets.keys()]
# table = widgets.VBox(table, layout={'padding': "5px 10px 0px 0px", 'align-content':'center'})

table = widgets.SelectMultiple(
    options=list(assets.keys()),
    value=list(assets.keys()),
    disabled=False,
    layout={'padding': "12px 20px 0px 0px"}
)

text = widgets.Textarea(
    value=description,
    placeholder='Type something',
#     description='String:',
    layout={'width': '70%',
            'height': '90px',
            'padding': "12px 20px 0px 0px"},
    disabled=False
)

title = widgets.HTML(
    value=ca.datasets._repr_html_(),
    placeholder='',
    description='',
    layout={'padding': "5px 10px 0px 0px"}
)

items = [title, table, text]
# widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))
widgets.Box(items, layout={"border":"50px"})

Box(children=(HTML(value='<div>\n<style scoped>\n    .dataframe tbody tr th:only-of-type {\n        vertical-a…

'<table>\n<tbody>\n<tr><td>Sun  </td><td style="text-align: right;">696000</td><td style="text-align: right;">   1.9891e+09</td></tr>\n<tr><td>Earth</td><td style="text-align: right;">  6371</td><td style="text-align: right;">5973.6       </td></tr>\n<tr><td>Moon </td><td style="text-align: right;">  1737</td><td style="text-align: right;">  73.5       </td></tr>\n<tr><td>Mars </td><td style="text-align: right;">  3390</td><td style="text-align: right;"> 641.85      </td></tr>\n</tbody>\n</table>'